Prepare data

In [5]:
# import pandas as pd
# import numpy as np

# n1, n2, n3, n4, n5 = 8, 2, 2, 63, 2
# filename_pums = 'data/franklin/franklin_pums10.csv'

# # prepare pums
# pums = pd.read_csv(filename_pums)
# pumaids = [*set(pums['PUMA'].to_list())]
# pumaids.sort()

# df = pums.groupby(['PUMA', 'RAC1P', 'SEX']).size()

# with open('data/franklin/franklin_r2s2.csv', 'w') as f:
#     for id in pumaids:
#         cnt = df.loc[(id, 2, 2)] # black females
#         f.write(str(id) + ',' + str(cnt) + '\n')

In [19]:
import geopandas as gpd
import pandas as pd

gdf_hh = gpd.read_file('data/franklin/franklin_households_all.shp')
df_puma = pd.read_csv('data/franklin/franklin_r2s2.csv')
df_puma['PUMAID'] = df_puma['PUMAID'].astype(str).str.zfill(5)

frames = []
for idx, row in df_puma.iterrows():
    pumaid = row['PUMAID']
    cnt = row['COUNT']
    gdf = gdf_hh[gdf_hh['PUMACE10'] == pumaid]
    sample = gdf.sample(n=cnt)
    frames.append(sample)

results = pd.concat(frames)
results.to_file('data/franklin/franklin_sample.shp')

Read data

In [1]:
import geopandas as gpd
from shapely.geometry import Point

gdf1 = gpd.read_file('data/franklin/franklin_sample.shp')
gdf2 = gpd.read_file('data/franklin/franklin_households_all.shp')

c:\Users\Yue\AppData\Local\Programs\Python\Python38\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


Model input

In [9]:
from sklearn.neighbors import BallTree
import pickle

k_neighbors = 31
def get_nearest(src_points, candidates, k_neighbors=10):
    """
    Find nearest neighbors for all source points from a set of candidate points
    modified from: https://automating-gis-processes.github.io/site/notebooks/L3/nearest-neighbor-faster.html
    """
    
    # Create tree from the candidate points
    tree = BallTree(candidates, leaf_size=15, metric='euclidean')

    # Find closest points and distances
    distances, indices = tree.query(src_points, k=k_neighbors)

    # Return indices and distances
    return indices, distances

in_pts = [(x,y) for x,y in zip(gdf1.geometry.x , gdf1.geometry.y)]
qry_pts =  [(x,y) for x,y in zip(gdf2.geometry.x , gdf2.geometry.y)]
X, X_dis = get_nearest(in_pts, qry_pts, k_neighbors)
pickle.dump(X, open('data/franklin/franklin_buff_k' + str(k_neighbors-1) + '.pickle', "wb"))
pickle.dump(X_dis, open('data/franklin/franklin_buff_dis_k' + str(k_neighbors-1) + '.pickle', "wb"))

X, X_dis

(array([[ 22654, 119781,  95314, ...,  97428,  27877,  95629],
        [ 48170,  70589,  45514, ...,  27543,  64190,  11695],
        [217070, 185480, 316065, ..., 396085, 321488, 427653],
        ...,
        [ 89941,  37714,  85889, ...,  62263,  72847,  29326],
        [253103, 432025, 192989, ..., 109124, 301018, 168170],
        [369054, 172644, 276628, ..., 137615, 424484, 238331]], dtype=int64),
 array([[  0.        ,  15.81209221,  27.85561867, ..., 117.21184408,
         123.63197443, 124.57203299],
        [  0.        ,  26.45222488,  29.18357645, ..., 123.79526372,
         125.11478124, 126.66514572],
        [  0.        ,   9.97868952,  13.18986719, ...,  79.08705572,
          79.1685651 ,  79.54023927],
        ...,
        [  0.        ,  33.00353501,  51.66590177, ..., 130.30021713,
         130.824802  , 133.23307524],
        [  0.        ,  22.88552008,  23.6435183 , ..., 130.59100028,
         130.59100028, 131.78119595],
        [  0.        ,  23.99601557,  27.

In [3]:
import pickle
import pandas as pd

k_neighbors = 10
with open('data/franklin/franklin_buff_k' + str(k_neighbors) + '.pickle', "rb") as f:
    object = pickle.load(f)
    
df = pd.DataFrame(object)
df.to_csv(r'data/franklin/franklin_buff_k' + str(k_neighbors) + '.csv')

In [3]:
# epsilon
eps = 0.01

# number of sample households
M = len(gdf1)

Optimization

In [4]:
from gurobipy import Model, GRB, LinExpr, quicksum
import numpy as np
import csv

with open('data/franklin/franklin_prob.csv', 'w', newline='') as f1:
    wr1 = csv.writer(f1)
    with open('data/franklin/franklin_prob_obj.csv', 'w', newline='') as f2:
        wr2 = csv.writer(f2)

        for k in range(M):
            # initialize model
            m = Model('td')


            # add objective function
            obj = LinExpr()

            # add decision variables and objective function
            p = {}     ## decision vairable
            d = np.zeros((k_neighbors, k_neighbors))
            for i in range(k_neighbors):
                for j in range(k_neighbors):
                    if j != i:
                    # objective
                        d[i, j] = gdf2.iloc[X[k, i]].geometry.distance(gdf2.iloc[X[k, j]].geometry)
                        p[i, j] = m.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1, name="p_%d_%d"%(i, j))
                        obj += p[i, j] * d[i, j]

            # add constraints
            for i1 in range(k_neighbors):
                for j in range(k_neighbors):
                    for i2 in range(k_neighbors):
                        if j != i1 and j != i2:
                            m.addConstr(p[i1, j] <= np.exp(eps * d[i1, i2]) * p[i2, j])
            for i in range(k_neighbors):
                tmp = [j for j in range(k_neighbors) if j != i]
                m.addConstr(quicksum(p[i, j] for j in tmp) == 1)
            
            m.setObjective(obj, GRB.MINIMIZE)

            m.update()
            m.optimize()

            # write prob values
            prob_values = [k]
            var_values = [var.X for var in m.getVars() if 'p_0_' == str(var.VarName[0:4])]
            prob_values.extend(var_values)
            wr1.writerow(prob_values)

            # write objective values
            obj = m.getObjective().getValue()
            wr2.writerow([k, obj])
            print(k)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-23
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: AMD Ryzen 5 5600X 6-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 820 rows, 90 columns and 1530 nonzeros
Model fingerprint: 0x32e91e47
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [2e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 90 rows and 0 columns
Presolve time: 0.01s
Presolved: 730 rows, 90 columns, 1530 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8487720e+02   6.200000e+01   0.000000e+00      0s
Extra simplex iterations after uncrush: 3
     132    4.1617484e+02   0.000000e+00   0.000000e+00      0s

Solved in 132 iterations and 0.02 seconds (0.00 work units)
Optimal objective  4.161748377e+02
0
Gu

KeyboardInterrupt: 

Impacts of input parameters

In [56]:
import pandas as pd

k_neighbors_all = [10, 20, 30]
eps_all = [0.1, 0.01, 0.001, 0.0001]

with open('data/franklin/sols/franklin_prob_obj_all.csv', 'w') as fw:
    fw.write('eps,k,sql\n')
    fw.flush()

    for eps in eps_all:
        for k_neighbors in k_neighbors_all:
            obj = pd.read_csv('data/franklin/sols/franklin_prob_obj_eps' + str(eps) + "_k" + str(k_neighbors) + '.csv', header=None, index_col=0)
            obj_mean = obj[1].mean()
            fw.write(str(eps) + ',' + str(k_neighbors) + ',' + str(obj_mean) + '\n')

In [27]:
import pandas as pd
import numpy as np
import random

k_neighbors_all = [10, 20, 30]
eps_all = [0.1, 0.01, 0.001, 0.0001]

for k_neighbors in k_neighbors_all:
    for eps in eps_all:
        masked_locs = []
        buffer = pickle.load(open('data/franklin/franklin_buff_k' + str(k_neighbors) + '.pickle', "rb"))
        prob = pd.read_csv('data/franklin/sols/franklin_prob_eps' + str(eps) + "_k" + str(k_neighbors) + '.csv', header=None, index_col=0)

        for idx, row in prob.iterrows():
            locs = buffer[idx]
            locs = np.delete(locs, 0)
            x = random.choices(locs, weights=tuple(row.tolist()), k=1)[0]
            masked_locs.append(x)
        gdf = gdf2.iloc[masked_locs]
        gdf.to_file('data/franklin/sols/shp/franklin_masked_eps' + str(eps) + "_k" + str(k_neighbors) + '.shp')         

Comparative analysis with randomized location swapping

(1) Expected distance displaced

In [10]:
def random_edd(buffer_dis):
    edd_random = []
    for row in buffer_dis:
        edd_random.append(np.mean(row))
    return sum(edd_random) / len(edd_random)

def gm_edd(prob, buffer_dis):
    edd_gm = []
    for idx, row in prob.iterrows():
        edd_gm.append(np.dot(buffer_dis[idx], row.tolist()))
    return sum(edd_gm) / len(edd_gm)

In [11]:
import pandas as pd
import pickle
import numpy as np

k_neighbors_all = [10, 20, 30]
eps_all = [0.1, 0.01, 0.001, 0.0001]

with open('data/franklin/sols/franklin_edd_all.csv', 'w') as fw:
    fw.write('method,eps,k,edd\n')
    fw.flush()

    for k_neighbors in k_neighbors_all:
        buffer_dis = pickle.load(open('data/franklin/franklin_buff_dis_k' + str(k_neighbors) + '.pickle', "rb"))
        buffer_dis = np.delete(buffer_dis, 0, 1)

        # random
        edd_random = random_edd(buffer_dis)
        fw.write('random,,' + str(k_neighbors) + ',' + str(edd_random) + '\n')
        print('edd_random:', edd_random)
        
        # gm
        for eps in eps_all:
            prob = pd.read_csv('data/franklin/sols/franklin_prob_eps' + str(eps) + "_k" + str(k_neighbors) + '.csv', header=None, index_col=0)
            edd_gm = gm_edd(prob, buffer_dis)
            fw.write('gm,' + str(eps) + ',' + str(k_neighbors) + ',' + str(edd_gm) + '\n')
            print('edd_gm:', edd_gm)

edd_random: 43.88582553571073
edd_gm: 18.863580469408273
edd_gm: 26.96121594251074
edd_gm: 41.12635560596364
edd_gm: 43.62456836438404
edd_random: 62.1246197554136
edd_gm: 19.411952703583697
edd_gm: 34.06705612220797
edd_gm: 51.812218869608856
edd_gm: 61.12443555839775
edd_random: 76.6924851540253
edd_gm: 19.534956963771965
edd_gm: 42.49507453171577
edd_gm: 55.82398522218376
edd_gm: 74.38384392099407


(2) Average nearest neighbors 

In [18]:
from pointpats import PointPattern
import random
import numpy as np

def origin_ann(gdf):
    points = np.array(np.stack([gdf.geometry.x, gdf.geometry.y], axis=1))
    pp = PointPattern(points)
    return pp.mean_nnd


def random_ann(buffer, ann_origin, T=100):
    ann = []
    for t in range(T):
        masked_locs = []
        for locs in buffer:
            locs = np.delete(locs, 0)
            x = random.choice(locs)
            masked_locs.append(x)
        gdf = gdf2.iloc[masked_locs]
        points = np.array(np.stack([gdf.geometry.x, gdf.geometry.y], axis=1))
        pp = PointPattern(points)
        ann.append(pp.mean_nnd)
    return sum(ann) / len(ann) - ann_origin


def gm_ann(prob, buffer, ann_origin, T=100):
    ann = []
    for t in range(T):
        masked_locs = []
        for idx, row in prob.iterrows():
            locs = buffer[idx]
            locs = np.delete(locs, 0)
            x = random.choices(locs, weights=tuple(row.tolist()), k=1)[0]
            masked_locs.append(x)
        gdf = gdf2.iloc[masked_locs]
        points = np.array(np.stack([gdf.geometry.x, gdf.geometry.y], axis=1))
        pp = PointPattern(points)
        ann.append(pp.mean_nnd)
    return sum(ann) / len(ann) - ann_origin

In [19]:
import pandas as pd
import pickle

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

ann_origin = origin_ann(gdf1)
print('ann_origin:', ann_origin)

k_neighbors_all = [10, 20, 30]
eps_all = [0.1, 0.01, 0.001, 0.0001]
T = 100

with open('data/franklin/sols/franklin_ann_all.csv', 'w') as fw:
    fw.write('method,eps,k,ann\n')
    fw.write('origin,,' + str(ann_origin) + '\n')
    fw.flush()
    
    for k_neighbors in k_neighbors_all:
        buffer = pickle.load(open('data/franklin/franklin_buff_k' + str(k_neighbors) + '.pickle', "rb"))
        ann_random = random_ann(buffer, ann_origin, T)
        fw.write('random,,' + str(k_neighbors) + ',' + str(ann_random) + '\n')
        print('ann_random:', ann_random)
        
        for eps in eps_all:
            prob = pd.read_csv('data/franklin/sols/franklin_prob_eps' + str(eps) + "_k" + str(k_neighbors) + '.csv', header=None, index_col=0)
            ann_gm = gm_ann(prob, buffer, ann_origin, T)
            fw.write('gm,' + str(eps) + ',' + str(k_neighbors) + ',' + str(ann_gm) + '\n')
            print('ann_gm:', ann_gm)


ann_origin: 193.87017948022927
ann_random: -4.520564948909197
ann_gm: -1.3312523108770336
ann_gm: -4.486357144666215
ann_gm: -5.072479350270669
ann_gm: -4.538217193746192
ann_random: -6.751801306286069
ann_gm: -1.3272165283506183
ann_gm: -7.480518932720031
ann_gm: -9.22015204110619
ann_gm: -7.08152103281995
ann_random: -8.262555654157438
ann_gm: -1.9926870443501912
ann_gm: -10.415538137786541
ann_gm: -12.269262393306406
ann_gm: -9.74583606741038


(3) Cluster detection

In [14]:
from sklearn.cluster import DBSCAN
import numpy as np

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

X = np.array(np.stack([gdf1.geometry.x, gdf1.geometry.y], axis=1))
db = DBSCAN(eps=1000, min_samples=30).fit(X)
labels = db.labels_
gdf = gdf1.copy()
gdf['db_origin'] = labels
gdf.to_file('data/franklin/franklin_sample_db.shp')

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

Estimated number of clusters: 12
Estimated number of noise points: 1969


In [15]:
import random
from sklearn import metrics
from sklearn.cluster import DBSCAN
import numpy as np

def random_db(buffer, labels, T=100):
    precision_all, recall_all, f1_score_all = [], [], []
    for t in range(T):
        masked_locs = []
        for locs in buffer:
            locs = np.delete(locs, 0)
            x = random.choice(locs)
            masked_locs.append(x)
        gdf = gdf2.iloc[masked_locs]
        points = np.array(np.stack([gdf.geometry.x, gdf.geometry.y], axis=1))
        db = DBSCAN(eps=500, min_samples=5).fit(points)
        preds = db.labels_
        preds_binary = [0 if i == -1 else 1 for i in preds]

        precision = metrics.precision_score(labels, preds_binary, average='weighted')
        recall = metrics.recall_score(labels, preds_binary, average='weighted')
        f1_score = metrics.f1_score(labels, preds_binary, average='weighted')
        precision_all.append(precision)
        recall_all.append(recall)
        f1_score_all.append(f1_score)
    return sum(precision_all) / len(precision_all), sum(recall_all) / len(recall_all), sum(f1_score_all) / len(f1_score_all)


def gm_db(prob, buffer, labels, T=100):
    precision_all, recall_all, f1_score_all = [], [], []
    for t in range(T):
        masked_locs = []
        labels_all = []
        for idx, row in prob.iterrows():
            locs = buffer[idx]
            labels_all.append(labels[idx])
            locs = np.delete(locs, 0)
            x = random.choices(locs, weights=tuple(row.tolist()), k=1)[0]
            masked_locs.append(x)
        gdf = gdf2.iloc[masked_locs]
        points = np.array(np.stack([gdf.geometry.x, gdf.geometry.y], axis=1))
        db = DBSCAN(eps=500, min_samples=5).fit(points)
        preds = db.labels_
        preds_binary = [0 if i == -1 else 1 for i in preds]

        precision = metrics.precision_score(labels_all, preds_binary, average='weighted')
        recall = metrics.recall_score(labels_all, preds_binary, average='weighted')
        f1_score = metrics.f1_score(labels_all, preds_binary, average='weighted')
        precision_all.append(precision)
        recall_all.append(recall)
        f1_score_all.append(f1_score)
    return sum(precision_all) / len(precision_all), sum(recall_all) / len(recall_all), sum(f1_score_all) / len(f1_score_all)

In [ ]:
import pandas as pd
import pickle
from sklearn.cluster import DBSCAN
import numpy as np

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

X = np.array(np.stack([gdf1.geometry.x, gdf1.geometry.y], axis=1))
db = DBSCAN(eps=500, min_samples=5).fit(X)
labels = db.labels_
labels_binary = [0 if i == -1 else 1 for i in labels]

k_neighbors_all = [10, 20, 30]
eps_all = [0.1, 0.01, 0.001, 0.0001]
T = 100

with open('data/franklin/sols/franklin_db_all.csv', 'w') as fw:
    fw.write('method,eps,k,precision,recall,f1\n')
    fw.flush()
    
    for k_neighbors in k_neighbors_all:
        buffer = pickle.load(open('data/franklin/franklin_buff_k' + str(k_neighbors) + '.pickle', "rb"))
        db_random = random_db(buffer, labels_binary, T)
        fw.write('random,,' + str(k_neighbors) + ',' + str(db_random[0]) + ',' + str(db_random[1]) + ',' + str(db_random[2]) + '\n')
        print('db_random:', db_random[0], db_random[1], db_random[2])
        
        for eps in eps_all:
            prob = pd.read_csv('data/franklin/sols/franklin_prob_eps' + str(eps) + "_k" + str(k_neighbors) + '.csv', header=None, index_col=0)
            db_gm = gm_db(prob, buffer, labels_binary, T)
            fw.write('gm,' + str(eps) + ',' + str(k_neighbors) + ',' + str(db_gm[0]) + ',' + str(db_gm[1]) + ',' + str(db_gm[2]) + '\n')
            print('db_gm:', db_gm[0], db_gm[1], db_gm[2])

db_random: 0.9782313395024848 0.978051643192488 0.9781173252462707
db_gm: 0.9862498941382577 0.9861521909233183 0.9861880043108853
db_gm: 0.9839034746459698 0.9838028169014081 0.9838392297798748
db_gm: 0.978925934707189 0.9787480438184665 0.9788125875736075
db_gm: 0.9784058053223204 0.9782120500782469 0.9782809642965584
db_random: 0.9704898865973273 0.9702053990610325 0.9703110153906417
db_gm: 0.9862962502464236 0.9862050078247263 0.9862386050629278
db_gm: 0.9806194134083459 0.9805633802816901 0.9805810780496619
db_gm: 0.9738526977264977 0.9736189358372457 0.9737040814607077
db_gm: 0.9714842764313957 0.9712773865414708 0.9713533466920978
db_random: 0.9658658520104807 0.9656259780907664 0.9657200465476827
db_gm: 0.9860883199941423 0.9859831735472507 0.9860209554581607
db_gm: 0.975788194752191 0.9757159624413155 0.9757370606324326
db_gm: 0.9717049295015613 0.9715082159624419 0.971583918676541
db_gm: 0.9661990479658452 0.9659956964006261 0.9660749561193911
